### WikiEngine

In [21]:
%run wikidump.py

path_base = '/Users/harangju/Developer/data/wiki/'
name_xml = 'enwiki-20190801-pages-articles-multistream.xml.bz2'
name_index = 'enwiki-20190801-pages-articles-multistream-index.txt.bz2'
path_xml = path_base + name_xml
path_index = path_base + name_index
dump = WikiDump(path_xml, path_index)

### Exploring the wiki dump

In [2]:
dump.load_page('Portal:Physics/Topics')
dump.links[:5]

WikiDump: Loading index...
WikiDump: Loaded.
Fetch new xml.


['Classical physics', 'mechanics', 'optics', 'electricity', 'magnetism']

In [6]:
dump.load_page('Danielle Bassett')
dump.links[:5]

WikiDump: Loading index...
WikiDump: Loaded.
Fetch new xml.


['University of Pennsylvania',
 'Pennsylvania State University',
 'University of Cambridge',
 'Sloan Research Fellowship',
 'MacArthur fellowship']

In [7]:
dump.load_page('Matter', filter_top=True)
dump.links[5:10]

Fetch new xml.


['plasma (physics)', 'quartz', 'solid', 'water', 'liquid']

In [8]:
dump.load_page('Index of physics articles (P)')
dump.links[50:54]

Fetch new xml.


['Pandemonium effect', 'Panemone windmill', 'Panofsky Prize', 'Pantur Silaban']

In [28]:
# testing XML cache
%time page = dump.load_page('AccessibleComputing')
%time page = dump.load_page('Anarchism')
%time page = dump.load_page('Angola')

Fetch new.
CPU times: user 115 ms, sys: 2.64 ms, total: 118 ms
Wall time: 119 ms
Return cache.
CPU times: user 58.2 ms, sys: 244 µs, total: 58.5 ms
Wall time: 58.6 ms
Fetch new.
CPU times: user 514 ms, sys: 4.13 ms, total: 518 ms
Wall time: 522 ms


### Wikipedia hypernet traversal

In [ ]:
import networkx as nx

class WikiCrawler():
    @staticmethod
    def bfs(graph, dump, queue, depth_goal=1):
        depth = 0
        depth_num_items = len(queue)
        depth_inc_pending = False
        while queue:
            page = queue.pop(0)
            depth_num_items -= 1
            if depth_num_items == 0:
                depth += 1
                print('Depth: ' + str(depth))
                depth_inc_pending = True
            if dump.load_page(page, filter_top=True):
                for link in dump.links:
                    link = str(link).split('#')[0].capitalize()
                    if (page, link) not in graph.edges:
                        graph.add_edge(link, page)
                        queue.append(link)
            if depth_inc_pending:
                depth_num_items = len(queue)
                depth_inc_pending = False
            if depth == depth_goal:
                break

In [ ]:
graph = nx.DiGraph()
queue = ['Matter']
WikiCrawler.bfs(graph, dump, queue, 3)

In [ ]:
len(graph.nodes)

### Port to gephi

In [ ]:
nx.write_gexf(graph, path_base + 'graphs/matter_d3.gexf')

### Edge weights